# Allocation Algorithm 11

> **1.** Create a list of dataframes.  Each dataframe consists of an EI and all the children assigned to that EI in the real data

> **2.** Create a cluster center with is an average of these values for each dataframe that is then averaged with the EI values to create a weighted cluster center leaning towards the EI

> **3.** Assign children to clusters.  Children are in order based on the z-score of their distance to their closest EI in ascending order (Largest deviation to smallest deviation).  After an EI has 17 children, no more children are assigned to them to create somewhat even distribution

> **4.** Push allocation to MySQL table locationdataalgo6

In [1]:
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
)
    
cursor = connection.cursor()

query = ("Select ei_id, longitude, latitude from ei "
         "WHERE ei_id != '65d75e28e09e67e025e8cd3c'")
cursor.execute(query)

ei_ids = cursor.fetchall()

eiDFs = []

for ei in ei_ids:
    ei_group_list = []
    data = {}
    data["id"] = ei[0]
    data["longitude"] = ei[1]
    data["latitude"] = ei[2]
    ei_child_query = ("SELECT child_id_1, ei_id_1, driving_distance, driving_time "
                      "FROM locationdata "
                      f"WHERE ei_id_1 = '{ei[0]}' AND child_id_2 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(ei_child_query)
    ei_child = cursor.fetchall()
    
    for row2 in ei_child:
        # Add columns with prefixes
        data[f'{row2[0]}_distance'] = row2[2]
        data[f'{row2[0]}_time'] = row2[3]
        
    ei1_ei_query = ("SELECT ei_id_1, ei_id_2, driving_distance, driving_time "
                    "FROM locationdata "
                    f"WHERE ei_id_1 = '{ei[0]}' AND child_id_1 is NULL and child_id_2 is NULL")
    cursor.execute(ei1_ei_query)
    ei1_ei = cursor.fetchall()
    
    for row3 in ei1_ei:
        data[f'{row3[1]}_distance'] = row3[2]
        data[f'{row3[1]}_time'] = row3[3]
    
    ei_ei2_query = ("SELECT ei_id_1, ei_id_2, driving_distance, driving_time "
                    "FROM locationdata "
                    f"WHERE ei_id_2 = '{ei[0]}' AND child_id_1 is NULL and child_id_2 is NULL")
    cursor.execute(ei_ei2_query)
    ei_ei2 = cursor.fetchall()
    
    for row4 in ei_ei2:
        data[f'{row4[0]}_distance'] = row4[2]
        data[f'{row4[0]}_time'] = row4[3]
        
    ei_group_list.append(data)
    
    childQuery = ("SELECT child.child_id, child.longitude, child.latitude FROM locationdatareal ld "
                  "join child on child.child_id = ld.child_id_1 "
                  f"where child_id_2 is null and ei_id_2 is null and ld.assigned = 1 and ei_id_1 = '{ei[0]}'")
    
    cursor.execute(childQuery)
    assignedChild = cursor.fetchall()
    
    for child in assignedChild:
        data = {}
        data["id"] = child[0]
        data["longitude"] = child[1]
        data["latitude"] = child[2]
        child1_child_query = ("SELECT child_id_1, child_id_2, driving_distance, driving_time "
                              "FROM locationdata "
                              f"WHERE child_id_1 = '{child[0]}' AND ei_id_1 IS NULL AND ei_id_2 IS NULL")
        cursor.execute(child1_child_query)
        child1_child = cursor.fetchall()

        for row2 in child1_child:
            # Add columns with prefixes
            data[f'{row2[1]}_distance'] = row2[2]
            data[f'{row2[1]}_time'] = row2[3]

        child_child2_query = ("SELECT child_id_1, child_id_2, driving_distance, driving_time "
                              "FROM locationdata "
                              f"WHERE child_id_2 = '{child[0]}' AND ei_id_1 IS NULL AND ei_id_2 IS NULL")
        cursor.execute(child_child2_query)
        child_child2 = cursor.fetchall()

        for row3 in child_child2:
            data[f'{row3[0]}_distance'] = row3[2]
            data[f'{row3[0]}_time'] = row3[3]

        child_ei_query = ("SELECT child_id_1, ei_id_1, driving_distance, driving_time "
                          "FROM locationdata "
                          f"WHERE child_id_1 = '{child[0]}' AND child_id_2 IS NULL AND ei_id_2 IS NULL")
        cursor.execute(child_ei_query)
        child_ei = cursor.fetchall()

        for row4 in child_ei:
            data[f'{row4[1]}_distance'] = row4[2]
            data[f'{row4[1]}_time'] = row4[3]

        ei_group_list.append(data)
    eiDataFrame = pd.DataFrame(ei_group_list)
    eiDataFrame = eiDataFrame.fillna(0)
    eiDFs.append(eiDataFrame)

In [3]:
clusterCenters = pd.DataFrame()
for df in eiDFs:
    for index, row in df.iterrows():
        ei_row = row
        ei_row = ei_row.to_frame()
        ei_row = ei_row.transpose()
        break
    id_value = df.loc[0, "id"]
    df = df.drop(columns=["id"])
    new_row = df.mean()
    new_row = new_row.to_frame()
    new_row = new_row.transpose()
    
    meanDF = pd.concat([ei_row, new_row], ignore_index = True)
    meanDF = meanDF.drop(columns=["id"])
    clusterCenterPoint = meanDF.mean()
    clusterCenterPoint["id"] = id_value
    clusterCenterPoint["assignment_count"] = 0
    clusterCenterPoint = clusterCenterPoint.to_frame()
    clusterCenterPoint = clusterCenterPoint.transpose()
    clusterCenters = pd.concat([clusterCenters, clusterCenterPoint], ignore_index = True)

print(clusterCenters)

    longitude   latitude 65d8d05fe09e67e025e8cda9_distance  \
0  -82.376027  36.333302                           25.9935   
1   -82.03539  36.629112                         64.636575   
2  -82.957477  36.238022                          23.76747   
3  -82.442713  36.305203                         21.662037   
4  -82.055566  36.424277                           48.3988   
5  -82.547454  36.521234                         43.616834   
6    -82.6565  36.475722                         41.236624   
7  -82.532677  36.276078                         17.412283   
8  -82.214112   36.57496                         53.418653   
9  -82.314368  36.321829                           27.4471   
10  -82.29029  36.345866                         33.175034   
11 -82.548537  36.532149                         41.636939   

   65d8d05fe09e67e025e8cda9_time 65d8d732e09e67e025e8cdb6_distance  \
0                      36.295597                         25.889342   
1                      76.202842                     

In [4]:
query = ("Select child_id_1, child.Longitude, child.Latitude, ((min(driving_distance) - avg(driving_distance)) / std(driving_distance)) as z_score "
         "from locationdata ld "
         "join child on child.child_id = ld.child_id_1 "
         "where child_id_2 is NULL and ei_id_2 is NULL "
         "group by child_id_1 "
         "order by z_score asc")
cursor.execute(query)

child_ids = cursor.fetchall()

data_list = []
for row in child_ids:
    data = {}
    data["id"] = row[0]
    data["longitude"] = row[1]
    data["latitude"] = row[2]
    child1_child_query = ("SELECT child_id_1, child_id_2, driving_distance, driving_time "
                          "FROM locationdata "
                          f"WHERE child_id_1 = '{row[0]}' AND ei_id_1 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(child1_child_query)
    child1_child = cursor.fetchall()
    
    for row2 in child1_child:
        # Add columns with prefixes
        data[f'{row2[1]}_distance'] = row2[2]
        data[f'{row2[1]}_time'] = row2[3]
        
    child_child2_query = ("SELECT child_id_1, child_id_2, driving_distance, driving_time "
                          "FROM locationdata "
                          f"WHERE child_id_2 = '{row[0]}' AND ei_id_1 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(child_child2_query)
    child_child2 = cursor.fetchall()
    
    for row3 in child_child2:
        data[f'{row3[0]}_distance'] = row3[2]
        data[f'{row3[0]}_time'] = row3[3]
    
    child_ei_query = ("SELECT child_id_1, ei_id_1, driving_distance, driving_time "
                      "FROM locationdata "
                      f"WHERE child_id_1 = '{row[0]}' AND child_id_2 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(child_ei_query)
    child_ei = cursor.fetchall()
    
    for row4 in child_ei:
        data[f'{row4[1]}_distance'] = row4[2]
        data[f'{row4[1]}_time'] = row4[3]
        
    data_list.append(data)

childDataFrame = pd.DataFrame(data_list)
childDataFrame = childDataFrame.fillna(0)
childDataFrame = childDataFrame.sample(frac=1)
childDataFrame.reset_index(drop=True, inplace=True)
childDataFrame["assigned"] = ""

print(childDataFrame)

                           id  longitude   latitude  \
0    65e6049b38f826f292672032 -82.554146  36.559986   
1    65d8da66e09e67e025e8cdbd -82.457680  36.297832   
2    65e6044938f826f292672031 -82.559898  36.563400   
3    65d8b217e09e67e025e8cd75 -81.940796  36.422821   
4    65d77c78e09e67e025e8cd5e -82.369080  36.349117   
..                        ...        ...        ...   
191  65e5e11438f826f292672018 -82.377785  36.294498   
192  65d8d4b3e09e67e025e8cdb1 -82.237961  36.495651   
193  65d8c331e09e67e025e8cd93 -82.995255  36.356022   
194  65d76d38e09e67e025e8cd4e -83.244423  36.348007   
195  65d8c4cde09e67e025e8cd96 -82.661781  36.572487   

     65e5cfc338f826f292671ff3_distance  65e5cfc338f826f292671ff3_time  \
0                              43.0553                        47.7105   
1                              25.9770                        37.2065   
2                              43.1526                        48.5335   
3                              28.2632         

In [5]:
childDataFrameOrdered = childDataFrame.sort_index(axis=1)
clusterCentersOrdered = clusterCenters.sort_index(axis=1)

In [6]:
def euclidean_distance(childRow, eiRow):
    child_values = childRow.drop(['id', 'assigned']).values
    ei_values = eiRow.drop(['id', 'assignment_count']).values
    return np.sqrt(np.sum((child_values - ei_values) ** 2))

In [7]:
childDataFrameAllocate = childDataFrameOrdered.copy()
for child_index, child_row in childDataFrameOrdered.iterrows():
    if childDataFrameAllocate.loc[child_index, "assigned"] == "":
        availableClusters = clusterCentersOrdered[clusterCentersOrdered['assignment_count'] < 17]
        eiDF = pd.DataFrame(columns=["id", "euclidean_distance"])
        for cluster_index, cluster_row in availableClusters.iterrows():
            e_value = euclidean_distance(child_row, cluster_row)
            eiDF = pd.concat([eiDF, pd.DataFrame({"id": [cluster_row["id"]], "euclidean_distance": [e_value]})], ignore_index=True)
        minValue = eiDF["euclidean_distance"].min()
        minIndex = eiDF["euclidean_distance"].idxmin()
        ei_with_min = eiDF.at[minIndex, "id"]

        same_location_children = childDataFrameOrdered[(childDataFrameOrdered["longitude"] == child_row["longitude"]) & 
                                                      (childDataFrameOrdered["latitude"] == child_row["latitude"])]
        childDataFrameOrdered.loc[same_location_children.index, 'assigned'] = ei_with_min
        
        rows_with_ei_with_min = childDataFrameOrdered[childDataFrameOrdered['assigned'] == ei_with_min]
        count_ei_with_min = len(rows_with_ei_with_min)
        
        clusters_index = clusterCentersOrdered[clusterCentersOrdered['id'] == ei_with_min].index
        clusterCentersOrdered.at[clusters_index[0], 'assignment_count'] = count_ei_with_min
    else:
        print("child already assigned")
    childDataFrameAllocate = childDataFrameOrdered.copy()

print(clusterCentersOrdered)

child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
   65d75a2ce09e67e025e8cd30_distance 65d75a2ce09e67e025e8cd30_time  \
0                           2.810574                      5.541683   
1                          41.816378                      49.47485   
2                           40.46475                      51.42384   
3                           6.235271                     11.843229   
4                          26.926075                     42.232767   
5                          21.692315                     27.335712   
6                          28.309241              

In [8]:
for index, cluster in clusterCentersOrdered.iterrows():
    cluster_id = cluster["id"]
    rows_with_ei_with_min = childDataFrameOrdered[childDataFrameOrdered['assigned'] == cluster_id]
    count_ei_with_min = len(rows_with_ei_with_min)
    print(f"children assigned to this ei: {count_ei_with_min}")

children assigned to this ei: 17
children assigned to this ei: 9
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17


In [9]:
for index, child in childDataFrameOrdered.iterrows():
    child_id = child["id"]
    ei_id = child["assigned"]
    updateQuery = ("Update locationdataalgo11 "
                   "SET assigned = 1 "
                   f"WHERE child_id_1 = '{child_id}' and ei_id_1 = '{ei_id}' and child_id_2 is NULL and ei_id_2 is NULL")
    cursor.execute(updateQuery)
    connection.commit()
    print("Update query pushed")

Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
